In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive


In [0]:
import numpy as np

In [0]:
trainImages = np.load('./diabetic-retinopathy-resized/0x1/trainImages.npy')
testImages = np.load('./diabetic-retinopathy-resized/0x1/testImages.npy')
y_train = np.load('./diabetic-retinopathy-resized/0x1/y_train.npy')
y_test = np.load('./diabetic-retinopathy-resized/0x1/y_test.npy')

In [4]:
print(trainImages.shape)
print(testImages.shape)
print(y_train.shape)
print(y_test.shape)

(25650, 256, 256, 3)
(6413, 256, 256, 3)
(25650, 2)
(6413, 2)


In [5]:
print(trainImages.max())
print(testImages.max())

255
255


In [6]:
from tensorflow.keras import datasets, layers, models

In [0]:
def convolutional_model():
    
    # create model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (5, 5), strides=(1, 1), activation='relu', input_shape=(256, 256, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (5,5), strides = (1,1), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding='same'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(128, (5,5), strides = (1,1), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding='same'))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(2, activation='softmax'))
    
    model.summary()
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    
    return model

In [8]:
model = convolutional_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 252, 252, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 64)      51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 61, 61, 64)        0         
_________________________________________

In [9]:
model.fit(trainImages, y_train, epochs=50, validation_data=(testImages, y_test))

Train on 25650 samples, validate on 6413 samples
Epoch 1/50
25650/25650 [==============================] - 52s 2ms/sample - loss: 1.9774 - acc: 0.7214 - val_loss: 0.5855 - val_acc: 0.7273
Epoch 2/50
25650/25650 [==============================] - 50s 2ms/sample - loss: 0.5903 - acc: 0.7235 - val_loss: 0.5903 - val_acc: 0.7273
Epoch 3/50
25650/25650 [==============================] - 50s 2ms/sample - loss: 0.5904 - acc: 0.7235 - val_loss: 0.5857 - val_acc: 0.7273
Epoch 4/50
25650/25650 [==============================] - 50s 2ms/sample - loss: 0.5898 - acc: 0.7235 - val_loss: 0.5850 - val_acc: 0.7273
Epoch 5/50
25650/25650 [==============================] - 50s 2ms/sample - loss: 0.5910 - acc: 0.7235 - val_loss: 0.5854 - val_acc: 0.7273
Epoch 6/50
25650/25650 [==============================] - 50s 2ms/sample - loss: 0.5906 - acc: 0.7235 - val_loss: 0.5858 - val_acc: 0.7273
Epoch 7/50
25650/25650 [==============================] - 50s 2ms/sample - loss: 0.5900 - acc: 0.7235 - val_loss: 0.5

In [10]:
scores = model.evaluate(testImages, y_test, verbose=0)
print('Accuracy: {}% \n Error: {}'.format(scores[1], 1 - scores[1])) 

Accuracy: 0.7288320660591125% 
 Error: 0.27116793394088745


In [11]:
from sklearn import metrics
y_pred = model.predict(testImages)
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
matrix

array([[4647,   17],
       [1722,   27]])

In [12]:
model.save("training_0x1_1.h5")
print("Saved model to disk")

Saved model to disk
